# WOfS Validation_Data-Clean  <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2),
[ga_ls8c_wofs_2_summary ](https://explorer.digitalearth.africa/ga_ls8c_wofs_2_summary)

## Background
The [Water Observations from Space (WOfS)](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs/about-wofs) is a derived product from Landsat 8 satellite observations as part of provisional Landsat 8 Collection 2 surface reflectance and shows surface water detected in Africa.
Individual water classified images are called Water Observation Feature Layers (WOFLs), and are created in a 1-to-1 relationship with the input satellite data. 
Hence there is one WOFL for each satellite dataset processed for the occurrence of water.

The data in a WOFL is stored as a bit field. This is a binary number, where each digit of the number is independantly set or not based on the presence (1) or absence (0) of a particular attribute (water, cloud, cloud shadow etc). In this way, the single decimal value associated to each pixel can provide information on a variety of features of that pixel. 
For more information on the structure of WOFLs and how to interact with them, see [Water Observations from Space](../Datasets/Water_Observations_from_Space.ipynb) and [Applying WOfS bitmasking](../Frequently_used_code/Applying_WOfS_bitmasking.ipynb) notebooks. 

## Description
This notebook explains how you can compile tables from Collect Earth Online tool from each partner institution and make them analysis-ready for WOfS analysis and accuracy assessment. 

The notebook demonstrates how to:

1. Load collected validation points as a list of observations each has a location and month
2. Data wrangling including cleaning the table, and mapping each point two twelve month observation 

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages

In [1]:
%matplotlib inline

import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from deafrica_plotting import display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard
from rasterio.mask import mask

### Connect to the datacube

In [2]:
dc = datacube.Datacube()

### Analysis parameters

In [3]:
#make sure that validation points have at least three columns : location (x,y), class, as well as 12 records for each observation  
#Path to the validation data points csv file 
CEO = '../Supplementary_data/Validation/CEO_3_RCMRD_2020-07-30.csv'

### Loading Dataset

In [4]:
#Read in the validation data csv
df = pd.read_csv(CEO, delimiter=",")
ground_truth = df.drop(['SAMPLE_ID','USER_ID','IMAGERY_TITLE','COLLECTION_TIME','ANALYSIS_DURATION','PL_PLOTID'], axis=1)

In [10]:
ground_truth.columns

Index(['PLOT_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES',
       'SENTINEL2MOSAICYEARMONTH',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS NOT OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, IMAGE WAS BAD?',
       'ENTER MONTHS[1-12] IN 2018, THAT YOU ARE UNSURE IF YOU OBSERVE WATER OR NOT? ',
       'CLASS', 'COMMENT'],
      dtype='object')

In [11]:
ground_truth.shape

(200, 12)

In [12]:
ground_truth = ground_truth.rename(columns={'WHAT IS THE FEATURE?':'CLASS','ENTER MONTHS[1-12] IN 2018, WATER WAS OBSERVED?':'WATER',
                                            'ENTER MONTHS[1-12] IN 2018, WATER WAS NOT OBSERVED?':'NO_WATER','ENTER MONTHS[1-12] IN 2018, IMAGE WAS BAD?':'BAD_IMAGE',
                                             'ENTER MONTHS[1-12] IN 2018, THAT YOU ARE UNSURE IF YOU OBSERVE WATER OR NOT? ':'NOT_SURE'})

In [13]:
ground_truth

,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2MOSAICYEARMONTH,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT
0,137387037,29.875854,2.178788,False,1,2018 - 2018,1-Dec,0,10,0,Open water - freshwater,Point is within the river channel
1,137387038,27.272168,0.893874,False,1,2018 - 2018,1-Dec,0,10,0,Open water - freshwater,Point is near the river bank
2,137387039,27.977967,0.489607,False,1,2018 - 2018,"1, 3-5, 7-12",0,"2, 3","2, 6",Open water - freshwater,Bad image month 2 & 6
3,137387040,28.071867,-3.271851,False,1,2018 - 2018,"1, 3, 4, 5, 6, 7, 12",8-Nov,2,2,Open water - freshwater,No image for February
4,137387041,28.708019,-5.872515,False,1,2018 - 2018,3-Sep,0,0,"1 - 2,10 -12",Open water - freshwater,It is difficult to know the month the water wa...
...,...,...,...,...,...,...,...,...,...,...,...,...
195,137387232,30.050283,-0.288776,False,1,2018 - 2018,"1,3,5-11",0,"2,4,12",0,Open water - marine,none
196,137387233,34.518275,-0.326637,False,1,2018 - 2018,1-Dec,0,0,0,Open water - marine,none
197,137387234,36.102439,-0.336985,False,1,2018 - 2018,"1-3,5-12",0,4,0,Open water - marine,none
198,137387235,31.875724,-0.358620,False,1,2019 - 2019,"1,2,4,6-8",0,"3,5,9,10-12",0,Open water - marine,none


In [14]:
#Converting column type to string if not already
ground_truth['NOT_SURE'] = ground_truth.NOT_SURE.astype(str)

In [15]:
cols = ['WATER','NO_WATER','BAD_IMAGE','NOT_SURE']
for col in cols:
    ground_truth[col] = ground_truth[col].str.replace('[','')
    ground_truth[col] = ground_truth[col].str.replace(']','')
    ground_truth[col] = ground_truth[col].str.replace('&','')
    ground_truth[col] = [''.join(c.split()) for c in ground_truth[col]]

In [16]:
#check whether any nan values in the dataframe and print it out against the column name 
# count_nan_in_df = ground_truth.isnull().sum()
# print (count_nan_in_df)

In [17]:
#replacing the name of months with their numerical values
replacements = { 'WATER': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'NO_WATER': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'BAD_IMAGE':{r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'}}

ground_truth.replace(replacements, regex=True, inplace=True)

In [18]:
ground_truth['SENTINEL2MOSAICYEARMONTH'] = ground_truth['SENTINEL2MOSAICYEARMONTH'].str.replace('2019-2019','2018-2018')

In [19]:
def split_str(row, newtable):
#check each row for No-WATER info an update the water column 
    monthstr=row['NO_WATER']
    if monthstr!='0'and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='0'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow)
#check each row for water info 
    monthstr=row['WATER']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='1'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow)  # update index / ignore original index
#check each row for bad image 
    monthstr=row['BAD_IMAGE']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='2'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow) 
    monthstr=row['NOT_SURE']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='3'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow) 
                
    return newtable

In [20]:
#count_nan_in_df = ground_truth.isnull().sum()
#print (count_nan_in_df)

In [21]:
#Making an empty dataframe
result = pd.DataFrame()

In [22]:
for irow in range(len(ground_truth)):
    result=split_str(ground_truth.iloc[irow], result)
    result.update(result)

In [23]:
result.shape
#result.loc[13]#this shows all the table 

(2507, 14)

In [24]:
indexNames = result[result.duplicated(['LAT', 'LON','MONTH'], keep=False)]
indexNames.shape

(274, 14)

In [25]:
result = result[['PLOT_ID', 'LON', 'LAT','FLAGGED','ANALYSES','SENTINEL2YEAR', 'WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS', 'COMMENT', 'MONTH','WATERFLAG']]

In [26]:
# indexNames = result[result.duplicated(['LAT', 'LON','MONTH'], keep=False) & (result['WATERFLAG'] <= '1') & (result['NOT_SURE']!='0')].index
# result.drop(indexNames , inplace=True)

In [27]:
#result

In [144]:
#group by PLOT Id and then do the estimation of the row number for each plot number in column Month  
count = result.groupby(['PLOT_ID'])['MONTH'].count()
count.to_csv('../Supplementary_data/Validation/Refined/CEO_1_RCMRD_count.csv')

In [28]:
#save the dataframe as csv file 
result.to_csv('../Supplementary_data/Validation/Refined/NewAnalysis/CEO_3_RCMRD_2020-07-30.csv')

In [69]:
#joining dataframes together and extract one csv for each partner institution 
DF = glob.glob('../Supplementary_data/Validation/Refined/*_RCMRD_*.csv')
frame = []
for d in DF: 
    f = pd.read_csv(d,delimiter=",")
    frame.append(f)
out = pd.concat(frame)
out.to_csv('../Supplementary_data/Validation/Refined/CEO_RCMRD_2020-07-30.csv')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** January 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)